In [91]:
import pymongo
import json
import pandas as pd
import time
import re

In [92]:
client=pymongo.MongoClient()

In [93]:
client.list_database_names()

['admin', 'config', 'local', 'pollution', 'twitterdb']

In [94]:
db=client.pollution
db.collection_names()

['tweetcollection']

In [95]:
db.tweetcollection.find({"place":{"$ne": None}}).count() # with place tweets

24995

In [96]:
db.tweetcollection.find({"geo":{"$ne": None}}).count() # with exact location

1013

In [97]:
pollution_pattern="pollution"

In [98]:
category_0=["air #pollution","空气污染","#airpollution","वायु प्रदुषण","la pollution de l'air","la #pollution","air pollution","l'air #pollution"]

In [99]:
category_0

['air #pollution',
 '空气污染',
 '#airpollution',
 'वायु प्रदुषण',
 "la pollution de l'air",
 'la #pollution',
 'air pollution',
 "l'air #pollution"]

In [100]:
regx = re.compile(r'('+'|'.join(category_0)+')', re.IGNORECASE)
regx

re.compile(r"(air #pollution|空气污染|#airpollution|वायु प्रदुषण|la pollution de l'air|la #pollution|air pollution|l'air #pollution)",
re.IGNORECASE|re.UNICODE)

In [101]:
withlocation=list(db.tweetcollection.find({"geo":{"$ne": None},"text":{"$regex":pollution_pattern}}))

In [102]:
len(withlocation)

11

In [103]:
withlocation[1]

{'_id': 994691118179602432,
 'contributors': None,
 'coordinates': {'coordinates': [114.15893555, 22.35388613], 'type': 'Point'},
 'created_at': 'Thu May 10 21:30:17 +0000 2018',
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [{'display_url': 'dlvr.it/QSRjd7',
    'expanded_url': 'http://dlvr.it/QSRjd7',
    'indices': [94, 117],
    'url': 'https://t.co/fqKCIvjV1z'}],
  'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'filter_level': 'low',
 'geo': {'coordinates': [22.35388613, 114.15893555], 'type': 'Point'},
 'id_str': '994691118179602432',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': {'attributes': {},
  'bounding_box': {'coordinates': [[[114.141298, 22.344083],
     [114.141298, 22.439185],
     [114.25441, 22.439185],
     [114.25441, 22.344083]]],
   'type': 'Polygon'},
  'country':

In [106]:
withplace=list(db.tweetcollection.find({"place":{"$ne": None}}))

In [108]:
withplace[0]

{'_id': 994256948102021121,
 'contributors': None,
 'coordinates': None,
 'created_at': 'Wed May 09 16:45:03 +0000 2018',
 'display_text_range': [0, 76],
 'entities': {'hashtags': [],
  'media': [{'display_url': 'pic.twitter.com/xu2PMQKcrj',
    'expanded_url': 'https://twitter.com/ValleyGreenTeam/status/994256948102021121/photo/1',
    'id': 994256937708503041,
    'id_str': '994256937708503041',
    'indices': [77, 100],
    'media_url': 'http://pbs.twimg.com/media/DcxPasOVQAEqUPY.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DcxPasOVQAEqUPY.jpg',
    'sizes': {'large': {'h': 1536, 'resize': 'fit', 'w': 2048},
     'medium': {'h': 900, 'resize': 'fit', 'w': 1200},
     'small': {'h': 510, 'resize': 'fit', 'w': 680},
     'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
    'type': 'photo',
    'url': 'https://t.co/xu2PMQKcrj'}],
  'symbols': [],
  'urls': [],
  'user_mentions': [{'id': 772369988,
    'id_str': '772369988',
    'indices': [0, 10],
    'name': 'Dibs',
    

In [112]:
def get_retweetid(tweet):
    try:
        return(tweet["retweeted_status"]["id"])
    except: 
        return None
    
def get_retweet_username(tweet):
    try:
        return(tweet["retweeted_status"]["user"]["name"])
    except: 
        return None
def get_retweet_followers_count(tweet):
    try:
        return(tweet["retweeted_status"]["user"]["followers_count"])
    except: 
        return None
def get_retweet_fa(tweet):
    try:
        return(tweet["retweeted_status"]["favorite_count"])
    except: 
        return None
def get_retweet_retw(tweet):
    try:
        return(tweet["retweeted_status"]["retweet_count"])
    except: 
        return None
def get_retweet_rep(tweet):
    try:
        return(tweet["retweeted_status"]["reply_count"])
    except: 
        return None
    
def jsontopandas(tweets_data):
    tweets=pd.DataFrame()
    tweets['id'] = list(map(lambda tweet: tweet['_id'], tweets_data))
    tweets['longitude']
    tweets['place_country']=list(map(lambda tweet: tweet['place']['country'] if tweet['place']!= None else None, tweets_data))
    tweets['place_full_name']=list(map(lambda tweet: tweet['place']['full_name'] if tweet['place']!= None else None, tweets_data))
    tweets['text']=list(map(lambda tweet: tweet['text'], tweets_data))
    tweets["user_id"]=list(map(lambda tweet:tweet["user"]["id"],tweets_data))
    tweets["user_name"]=list(map(lambda tweet:tweet["user"]["name"],tweets_data))
    tweets["user_followers"]=list(map(lambda tweet:tweet["user"]["followers_count"],tweets_data))
    tweets['user_loc']=list(map(lambda tweet: tweet['user']['location'], tweets_data))
    tweets['lang']=list(map(lambda tweet: tweet['lang'], tweets_data))
    tweets['created_at']=list(map(lambda tweet: time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')), tweets_data))
    return tweets

In [ ]:
thistweet=jsontopandas(withplace)

In [124]:
thistweet.to_csv("../output/tweet_with_location.csv")

In [114]:
tweets_data=withplace

In [39]:
tweets=pd.DataFrame()
tweets['id'] = list(map(lambda tweet: tweet['_id'], tweets_data))
tweets['place_country']=list(map(lambda tweet: tweet['place']['country'] if tweet['place']!= None else None, tweets_data))
tweets['place_full_name']=list(map(lambda tweet: tweet['place']['full_name'] if tweet['place']!= None else None, tweets_data))
tweets['text']=list(map(lambda tweet: tweet['text'], tweets_data))
tweets["user_id"]=list(map(lambda tweet:tweet["user"]["id"],tweets_data))
tweets["user_name"]=list(map(lambda tweet:tweet["user"]["name"],tweets_data))
tweets["user_followers"]=list(map(lambda tweet:tweet["user"]["followers_count"],tweets_data))
tweets['user_loc']=list(map(lambda tweet: tweet['user']['location'], tweets_data))
tweets['lang']=list(map(lambda tweet: tweet['lang'], tweets_data))
tweets['created_at']=list(map(lambda tweet: time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')), tweets_data))

# tweets['retweet_count']=list(map(lambda tweet: tweet['retweet_count'] if tweet['retweet_count']!= None else None, tweets_data))
# tweets['reply_count']=list(map(lambda tweet: tweet['reply_count'] if tweet['reply_count']!= None else None, tweets_data))
# tweets['favorite_count']=list(map(lambda tweet: tweet['favorite_count'], tweets_data))

In [125]:
tweets.count()

id                 24995
place_country      24995
place_full_name    24995
text               24995
user_id            24995
user_name          24995
user_followers     24995
user_loc           20362
lang               24995
created_at         24995
dtype: int64

In [22]:
with open("../output/withplace.json","a") as f:
    for tweet in withplace:
        f.write(json.dumps(tweet))